In [1]:
PROJECT = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your PROJECT
BUCKET = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for Cloud MLE
TFVERSION = "1.14"                # TF version for CMLE to use

In [2]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
fi

# Create ML datasets by sampling using BigQuery

In [4]:
# Create SQL query using natality data after the year 2000
query_string = """
WITH
  CTE_hash_cols_fixed AS (
  SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
      WHEN day IS NULL AND wday IS NULL THEN 0
    ELSE
    CASE
      WHEN day IS NULL THEN wday
    ELSE
    wday
  END
  END
    AS date,
    IFNULL(state,
      "Unknown") AS state,
    IFNULL(mother_birth_state,
      "Unknown") AS mother_birth_state
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000)

SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING), CAST(date AS STRING), CAST(state AS STRING), CAST(mother_birth_state AS STRING)))) AS hashvalues
FROM
  CTE_hash_cols_fixed
"""

In [5]:
from google.cloud import bigquery
bq = bigquery.Client(project = PROJECT)

df = bq.query("SELECT hashvalues, COUNT(weight_pounds) AS num_babies FROM (" 
              + query_string + 
              ") GROUP BY hashvalues").to_dataframe()

print("There are {} unique hashvalues.".format(len(df)))
df.head()

There are 658107 unique hashvalues.


,hashvalues,num_babies
0,8439164539444335271,997
1,6608190415809312963,209
2,3820521621488099714,1617
3,4503554668502915020,689
4,7764133063593555444,812


In [6]:
sampling_percentages_query = """
WITH
  -- Get label, features, and column that we are going to use to split into buckets on
  CTE_hash_cols_fixed AS (
  SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
      WHEN day IS NULL AND wday IS NULL THEN 0
    ELSE
    CASE
      WHEN day IS NULL THEN wday
    ELSE
    wday
  END
  END
    AS date,
    IFNULL(state,
      "Unknown") AS state,
    IFNULL(mother_birth_state,
      "Unknown") AS mother_birth_state
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000),
  CTE_data AS (
  SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING), CAST(date AS STRING), CAST(state AS STRING), CAST(mother_birth_state AS STRING)))) AS hashvalues
  FROM
    CTE_hash_cols_fixed),
  -- Get the counts of each of the unique hashs of our splitting column
  CTE_first_bucketing AS (
  SELECT
    hashvalues,
    COUNT(*) AS num_records
  FROM
    CTE_data
  GROUP BY
    hashvalues ),
  -- Get the number of records in each of the hash buckets
  CTE_second_bucketing AS (
  SELECT
    MOD(hashvalues, {0}) AS bucket_index,
    SUM(num_records) AS num_records
  FROM
    CTE_first_bucketing
  GROUP BY
    MOD(hashvalues, {0})),
  -- Calculate the overall percentages
  CTE_percentages AS (
  SELECT
    bucket_index,
    num_records,
    CAST(num_records AS FLOAT64) / (
    SELECT
      SUM(num_records)
    FROM
      CTE_second_bucketing) AS percent_records
  FROM
    CTE_second_bucketing ),
  -- Choose which of the hash buckets will be used for training and pull in their statistics
  CTE_train AS (
  SELECT
    *,
    "train" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= 0
    AND bucket_index < {1}),
  -- Choose which of the hash buckets will be used for validation and pull in their statistics
  CTE_eval AS (
  SELECT
    *,
    "eval" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= {1}
    AND bucket_index < {2}),
  -- Choose which of the hash buckets will be used for testing and pull in their statistics
  CTE_test AS (
  SELECT
    *,
    "test" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= {2}
    AND bucket_index < {0}),
  -- Union the training, validation, and testing dataset statistics
  CTE_union AS (
  SELECT
    0 AS dataset_id,
    *
  FROM
    CTE_train
  UNION ALL
  SELECT
    1 AS dataset_id,
    *
  FROM
    CTE_eval
  UNION ALL
  SELECT
    2 AS dataset_id,
    *
  FROM
    CTE_test ),
  -- Show final splitting and associated statistics
  CTE_split AS (
  SELECT
    dataset_id,
    dataset_name,
    SUM(num_records) AS num_records,
    SUM(percent_records) AS percent_records
  FROM
    CTE_union
  GROUP BY
    dataset_id,
    dataset_name )
SELECT
  *
FROM
  CTE_split
ORDER BY
    dataset_id
"""

modulo_divisor = 100
train_percent = 80.0
eval_percent = 10.0

train_buckets = int(modulo_divisor * train_percent / 100.0)
eval_buckets = int(modulo_divisor * eval_percent / 100.0)

df = bq.query(sampling_percentages_query.format(modulo_divisor, train_buckets, train_buckets + eval_buckets)).to_dataframe()
df.head()

,dataset_id,dataset_name,num_records,percent_records
0,0,train,26080035,0.783845
1,1,eval,3639721,0.109393
2,2,test,3552158,0.106761


In [7]:
# Added every_n so that we can now subsample from each of the hash values to get approximately the record counts we want
every_n = 500

train_query = "SELECT * FROM ({0}) WHERE MOD(hashvalues, {1} * 100) < 80".format(query_string, every_n)
eval_query = "SELECT * FROM ({0}) WHERE MOD(hashvalues, {1} * 100) >= 80 AND MOD(hashvalues, {1} * 100) < 90".format(query_string, every_n)
test_query = "SELECT * FROM ({0}) WHERE MOD(hashvalues, {1} * 100) >= 90 AND MOD(hashvalues, {1} * 100) < 100".format(query_string, every_n)

train_df = bq.query(train_query).to_dataframe()
eval_df = bq.query(eval_query).to_dataframe()
test_df = bq.query(test_query).to_dataframe()

print("There are {} examples in the train dataset.".format(len(train_df)))
print("There are {} examples in the validation dataset.".format(len(eval_df)))
print("There are {} examples in the test dataset.".format(len(test_df)))

There are 18998 examples in the train dataset.
There are 1513 examples in the validation dataset.
There are 1389 examples in the test dataset.


# Preprocess data using Pandas

In [8]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashvalues
0,4.687028,True,15,1,36.0,5815559452861000023
1,7.500126,False,43,1,40.0,4450673194547000070
2,7.125340,False,44,1,39.0,7865574002928550049
3,10.000168,True,14,1,45.0,3721975748227700022
4,10.740921,False,43,1,42.0,3480083103445950001


In [9]:
train_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashvalues
count,18978.000000,18998.000000,18998.000000,18850.000000,1.899800e+04
mean,7.243111,27.782346,1.038162,38.639735,4.288488e+18
std,1.337393,6.195585,0.210196,2.594170,2.534495e+18
min,0.500449,13.000000,1.000000,17.000000,5.826385e+15
25%,6.572531,23.000000,1.000000,38.000000,1.861630e+18
50%,7.339189,28.000000,1.000000,39.000000,3.835475e+18
75%,8.062305,32.000000,1.000000,40.000000,6.784884e+18
max,12.500210,51.000000,4.000000,47.000000,9.210618e+18


In [10]:
import pandas as pd

def preprocess(df):
    # Clean up data
    # Remove what we don"t want to use for training
    df = df[df.weight_pounds > 0]
    df = df[df.mother_age > 0]
    df = df[df.gestation_weeks > 0]
    df = df[df.plurality > 0]

    # Modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"]))
    df["plurality"].replace(twins_etc, inplace = True)

    # Now create extra rows to simulate lack of ultrasound
    no_ultrasound = df.copy(deep = True)
    no_ultrasound.loc[no_ultrasound["plurality"] != "Single(1)", "plurality"] = "Multiple(2+)"
    no_ultrasound["is_male"] = "Unknown"

    # Concatenate both datasets together and shuffle
    return pd.concat([df, no_ultrasound]).sample(frac=1).reset_index(drop=True)

In [11]:
train_df = preprocess(train_df)
eval_df = preprocess(eval_df)
test_df = preprocess(test_df)

In [12]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashvalues
0,7.687519,Unknown,22,Single(1),38.0,8292885215117800033
1,0.562179,Unknown,20,Single(1),20.0,4896699230184800022
2,7.500126,Unknown,38,Single(1),38.0,2054179216119800044
3,6.686620,False,30,Single(1),39.0,895693915035850036
4,8.818490,Unknown,37,Single(1),40.0,1409348435509100014


In [13]:
train_df.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashvalues
37671,5.983346,False,39,Single(1),36.0,3835474516102300019
37672,7.374463,Unknown,28,Single(1),40.0,1861629504184950069
37673,5.813590,Unknown,34,Single(1),40.0,2221826412118750036
37674,7.438397,Unknown,28,Single(1),37.0,772843865200050032
37675,6.874013,False,36,Single(1),38.0,1494773057894250048


In [14]:
train_df.describe()

,weight_pounds,mother_age,gestation_weeks,hashvalues
count,37676.000000,37676.000000,37676.000000,3.767600e+04
mean,7.244695,27.767544,38.644973,4.277547e+18
std,1.335645,6.189406,2.576290,2.535502e+18
min,0.500449,13.000000,18.000000,5.826385e+15
25%,6.580799,23.000000,38.000000,1.861630e+18
50%,7.341393,28.000000,39.000000,3.825089e+18
75%,8.062305,32.000000,40.000000,6.784884e+18
max,12.500210,51.000000,47.000000,9.210618e+18


# Write to .csv files

In [15]:
columns = "weight_pounds,is_male,mother_age,plurality,gestation_weeks".split(',')
train_df.to_csv(path_or_buf = "train.csv", columns = columns, header = False, index = False)
eval_df.to_csv(path_or_buf = "eval.csv", columns = columns, header = False, index = False)
test_df.to_csv(path_or_buf = "test.csv", columns = columns, header = False, index = False)

In [16]:
%%bash
wc -l *.csv

   3000 eval.csv
   2764 test.csv
  37676 train.csv
  43440 total


In [17]:
%%bash
head *.csv

==> eval.csv <==
7.0437692708999995,Unknown,20,Single(1),28.0
6.4992274837599995,Unknown,30,Single(1),39.0
9.49971886958,Unknown,33,Single(1),39.0
8.99926953484,Unknown,19,Single(1),40.0
6.9225150268,True,22,Single(1),37.0
8.56275425608,Unknown,39,Single(1),39.0
6.2501051276999995,Unknown,23,Single(1),38.0
8.62448368944,True,29,Single(1),38.0
7.5618555866,False,29,Single(1),40.0
8.12623897732,True,26,Single(1),40.0

==> test.csv <==
8.811876612139999,Unknown,39,Single(1),41.0
9.4909003791,True,26,Single(1),40.0
7.3744626639,False,35,Single(1),40.0
6.4815905028,True,21,Single(1),37.0
6.4374980503999994,Unknown,21,Single(1),40.0
6.1244416383599996,False,23,Single(1),31.0
4.81269117946,Unknown,26,Single(1),40.0
6.87621795178,False,27,Single(1),40.0
8.437090766739999,False,25,Single(1),39.0
9.06320359082,Unknown,24,Single(1),40.0

==> train.csv <==
7.68751907594,Unknown,22,Single(1),38.0
0.5621787681,Unknown,20,Single(1),20.0
7.50012615324,Unknown,38,Single(1),38.0
6.686620406459999,False,

In [18]:
%%bash
tail *.csv

==> eval.csv <==
8.35331510718,False,18,Single(1),40.0
5.06181353552,False,37,Twins(2),35.0
8.1791499202,Unknown,19,Single(1),41.0
6.56316153974,False,25,Single(1),39.0
8.68841774542,Unknown,36,Single(1),40.0
6.93794738514,True,32,Single(1),39.0
9.78411518756,False,28,Single(1),38.0
1.06262810284,False,22,Single(1),22.0
9.294688965919999,False,26,Single(1),37.0
6.37576861704,Unknown,22,Single(1),35.0

==> test.csv <==
7.9807338844,Unknown,23,Single(1),39.0
5.6592662655399995,True,32,Single(1),39.0
7.6279942652,Unknown,41,Single(1),39.0
8.99926953484,Unknown,22,Single(1),41.0
7.936641432,Unknown,37,Single(1),40.0
7.1870697412,Unknown,37,Single(1),41.0
3.196702799,False,19,Single(1),29.0
8.68841774542,Unknown,26,Single(1),39.0
5.8135898489399995,Unknown,32,Single(1),38.0
7.81318256528,Unknown,19,Single(1),40.0

==> train.csv <==
7.2421853067,True,28,Single(1),38.0
8.4767739739,True,30,Single(1),40.0
7.06140625186,Unknown,19,Single(1),41.0
3.6927428884999998,False,30,Single(1),32.0
6.8122